In [1]:
import os
import sys
import sqlite3
import numpy as np
import pandas as pd
import geopandas as gp
import plotly.express as px

import pyet
import pyeto

In [2]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

GEO_DATA_PATH = "./assets/geo_data"
DATABASE_PATH = "./database/database.db"

In [3]:
conn = sqlite3.connect(DATABASE_PATH)

daily_data = pd.read_sql(sql='SELECT * FROM ground_data_daily', con=conn)
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
monthly_data = pd.read_sql(sql='SELECT * FROM ground_data_monthly', con=conn)
monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])
geoinfo = pd.read_sql(sql='SELECT * FROM ground_data_geoinfo', con=conn)

conn.close()

In [4]:
daily_geoinfo_data = pd.merge(daily_data, geoinfo, on='Station_ID', how='left')
monthly_geoinfo_data = pd.merge(monthly_data, geoinfo, on='Station_ID', how='left')

In [5]:
PET_Hargreaves = daily_geoinfo_data.set_index(['Date'])\
    .groupby(by="Station_ID")\
    .apply(lambda x: round(pyet.hargreaves(tmean=x['Temp_Mean'], tmax=x['Temp_Max'], tmin=x['Temp_Min'], lat=pyet.utils.deg_to_rad(x['Station_Latitude'].iloc[0])), 2), include_groups=False)\
    .stack(future_stack=True).reset_index()\
    .rename(columns={0: 'PET_Hargreaves'})

daily_geoinfo_data = daily_geoinfo_data.merge(PET_Hargreaves, on=['Date', 'Station_ID'], how='left')
daily_geoinfo_data

/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/pooya/w/DroughtMonitoringIran/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)

,Station_ID,Date,Temp_Max,Temp_Min,Temp_Mean,Precip,Wind_Speed,Pressure,Humidity_Max,Humidity_Min,Humidity,Sunshine,Station_Name,Province,Station_Latitude,Station_Longitude,Station_Elevation,PET_Hargreaves
0,40759,2006-01-01,11.2,-0.6,5.3,0.00,1.25000,1027.94,92.0,40.0,70.8750,7.8,Sari,Mazandaran,36.536,52.998,23.0,1.17
1,40759,2006-01-02,11.0,0.2,5.6,0.00,0.50000,1028.40,88.0,51.0,74.0000,6.3,Sari,Mazandaran,36.536,52.998,23.0,1.14
2,40759,2006-01-03,12.4,2.0,7.2,0.00,0.50000,1029.43,92.0,43.0,72.7500,4.5,Sari,Mazandaran,36.536,52.998,23.0,1.20
3,40759,2006-01-04,13.4,0.9,7.1,0.00,0.62500,1030.61,94.0,47.0,76.3750,8.1,Sari,Mazandaran,36.536,52.998,23.0,1.32
4,40759,2006-01-05,13.0,0.4,6.7,0.00,1.25000,1026.72,94.0,41.0,73.0000,8.0,Sari,Mazandaran,36.536,52.998,23.0,1.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169120,40719,2024-07-05,34.6,20.2,27.4,0.00,0.37500,1006.50,98.0,44.0,72.5000,12.5,Rasht,Gilan,37.323,49.624,-8.6,6.70
169121,40719,2024-07-06,32.2,21.8,27.0,0.00,0.87500,1006.42,100.0,56.0,79.7500,8.5,Rasht,Gilan,37.323,49.624,-8.6,5.63
169122,40719,2024-07-07,31.4,23.8,27.6,0.01,1.50000,1008.44,100.0,59.0,81.3750,8.7,Rasht,Gilan,37.323,49.624,-8.6,4.88
169123,40719,2024-07-08,32.5,23.6,28.0,0.00,1.25000,1009.80,95.0,54.0,74.6250,9.7,Rasht,Gilan,37.323,49.624,-8.6,5.32


In [6]:
min_count = 15
pet_monthly = (
    daily_geoinfo_data[['Station_ID', 'Date', 'PET_Hargreaves']]\
    .set_index('Date')
    .groupby(by='Station_ID')
    .resample(rule='ME')
    .agg(
        {
            'PET_Hargreaves': lambda x: x.sum(min_count=min_count),
        }
    )
).reset_index()

pet_monthly

,Station_ID,Date,PET_Hargreaves
0,40709,2006-01-31,23.76
1,40709,2006-02-28,30.78
2,40709,2006-03-31,55.38
3,40709,2006-04-30,70.81
4,40709,2006-05-31,103.12
...,...,...,...
5570,99361,2024-03-31,67.21
5571,99361,2024-04-30,88.16
5572,99361,2024-05-31,107.19
5573,99361,2024-06-30,128.74


In [7]:
conn = sqlite3.connect(DATABASE_PATH)

daily_geoinfo_data[['Station_ID', 'Date', 'PET_Hargreaves']].to_sql('pet_daily', conn, if_exists='replace', index=False)
pet_monthly.to_sql('pet_monthly', conn, if_exists='replace', index=False)

conn.commit()
conn.close()